In [ ]:
#################################################################################
# A set of helper utilities for Bitbucket, functionality supported
# The implementation uses standard rest APIs since the python bitbucket libraries 
# dont work well with cloud version.
#
# Features supported:
# 1. List all repos in a workspace + project
# 2. Update default reviewers (this is idempotent and can be called multiple times)
# 3. Setup branch restricitions (not idempotent and should be called ones, will fail on repeat)
#
# Getting started
# 1. Setup a bitbucket application password
# 2. Setup the environment variables within your pythong notebook, for example run below (do not enter quotes)
# ```
# %env BITBUCKET_USER=<your username>
# %env BITBUCKET_PASS=<application password>
# %env BITBUCKET_URL=https://api.bitbucket.org/
# %env BITBUCKET_WORKSPACE=inspiredbytech
# %env PROJECT_KEY=ABC
# %env USER_ID={xxxx-xxxx-xxxx-xxx-xx}
# ```
#################################################################################

from atlassian.bitbucket.cloud import Cloud
import os, json
import requests
import pandas as pd

username = os.getenv('BITBUCKET_USER', None)
password = os.getenv('BITBUCKET_PASS', None)
url = os.getenv('BITBUCKET_URL', None)
workspace = os.getenv('BITBUCKET_WORKSPACE', None)
project_key = os.getenv('PROJECT_KEY', None)
user_id = os.getenv('USER_ID', None)


In [ ]:

headers = {
    'Content-Type': 'application/json',
}
params = (
    ('key1', 'mykeyhere'),
)
url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/?q=project.key="{project_key}"&limit=1000&pagelen=100'


def get_repo(repo_name):
    url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_name}'
    rsp = requests.get(url,  headers=headers, auth=(username, password))
    print(json.dumps(rsp.json(), sort_keys=True, indent=4)) 
    
def get_repo_default_reviewers(repo_name):
    url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_name}/default-reviewers'
    rsp = requests.get(url,  headers=headers, auth=(username, password))
    print(json.dumps(rsp.json(), sort_keys=True, indent=4)) 

def put_repo_default_reviewers(repo_name, target_username_uuid):
    url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_name}/default-reviewers/{target_username_uuid}'
    print(url)
    rsp = requests.put(url, data = { 'target_username':target_username_uuid}, auth=(username, password))
    print(f'{rsp.status_code} {rsp.text} {rsp.reason}')

def get_repo_branch_restrictions(repo_name):
    url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_name}/branch-restrictions'
    print(url)
    rsp = requests.get(url,
        headers = headers, 
        auth=(username, password))
    print(json.dumps(rsp.json(), sort_keys=True, indent=4)) 

## to be completed
def post_repo_branch_restrictions(repo_name):
    restrictions = {
        'require_passing_builds_to_merge': {
            'kind': "require_passing_builds_to_merge",
            "pattern": "",
            "branch_match_kind": "branching_model",
            "branch_type": "development",
            "type": "branchrestriction",
            "value": 1
        },
        'restrict_merges': {
            "branch_match_kind": "branching_model",
            "branch_type": "development",
            "groups": [],
            "kind": "restrict_merges",
            "pattern": "",
            "type": "branchrestriction",
            "users": [
                {
                    "type": "user",
                    "uuid": f"{user_id}"
                }
            ],
        },
        'require_default_reviewer_approvals_to_merge': {
            "branch_match_kind": "branching_model",
            "branch_type": "development",
            "groups": [],
            "kind": "require_default_reviewer_approvals_to_merge",
            "pattern": "",
            "type": "branchrestriction",
            "value": 1
        },
        'require_approvals_to_merge': {
            "branch_match_kind": "branching_model",
            "branch_type": "development",
            "groups": [],
            "kind": "require_approvals_to_merge",
            "pattern": "",
            "type": "branchrestriction",
            "value": 1
        },
        'delete': {
            "branch_match_kind": "branching_model",
            "branch_type": "development",
            "groups": [],
            "kind": "delete",
            "pattern": "",
            "type": "branchrestriction",
            "users": [],
        },
        'force': {
            "branch_match_kind": "branching_model",
            "branch_type": "development",
            "groups": [],
            "kind": "force",
            "pattern": "",
            "type": "branchrestriction",
        },
        'push': {
            "branch_match_kind": "branching_model",
            "branch_type": "development",
            "groups": [],
            "kind": "push",
            "pattern": "",
            "type": "branchrestriction",
            "users": [],
        }
    }
    
    url = f'https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_name}/branch-restrictions'
    for key, value in restrictions.items():
        print(f'{repo_name} trying branch restriction {key}')
        rsp = requests.post(url,
            json = value,
            auth=(username, password))
        print(f'Status: {rsp.status_code} {rsp.text} {rsp.reason}')

def _main():
    headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
    repos_request = requests.get(url,  headers=headers,auth=(username, password))
    repos_json = repos_request.json()
    df = pd.DataFrame.from_dict(repos_json['values'])

    for repo in df['name']:
        #get_repo_default_reviewers(repo)
        #    put_repo_default_reviewers(repo, user_id)
        #    get_repo_branch_restrictions(repo)
        if repo == 'service':
            post_repo_branch_restrictions(repo)
_main()

In [ ]:

#
bitbucket = Cloud(
    url=url,
    username = username,
    password = password,
    cloud=True)

index = 0
project = bitbucket.workspaces.get(workspace).projects.get('XLO')

for r in project.repositories.each():
    print(f"Repo {r.name}")
    for dr in r.default_reviewers.each():
            print("   Default reviewer " + dr.nickname)
    for br in r.branch_restrictions.each():
        print(br)
    
#for r in project.repos.

In [ ]:
import json
from types import SimpleNamespace

# Parse JSON into an object with attributes corresponding to dict keys.
x = json.loads(r.text, object_hook=lambda d: SimpleNamespace(**d))
print(x[0])